### Load the data


In [1]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np
import sys
sys.path.insert(0, '../../base_codes')
sys.path.insert(0, '../../data_specific_codes')
sys.path.insert(0, '../../utility_codes')

from RE_init import *
from main_functions import *
from utility_functions import *
from SDDb_utility_functions import *

#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

based_dir = ''
file_input_postfix = ".csv"
input_name_prefix = 'SDDb_all_clean'#'SDDb_car_clean'#_relations_-1'#_filter(car)'
file_input_name = input_name_prefix + file_input_postfix



DATA_SET = 'sddb-car'
#output_dir = ""
#based_dir = based_dir + output_dir

def read_df_rel(based_dir, file_input_name):
    file_input = based_dir + file_input_name    
    ff = open(file_input)
    delim=","
    df = pd.read_csv(file_input,delimiter=delim,header=0)        
    return df

df_rels = read_df_rel(based_dir, file_input_name)

In [2]:
print len(df_rels)
df_rels.dropna(subset=["text"], how="all", inplace=True)
print len(df_rels)

33453
33448


In [3]:
a = df_rels["person id"].unique()
b = a.tolist()

In [4]:
len(b)

12560

In [5]:
c = [i.split(":")[0] for i in b]

In [6]:
c_unique = set(c)

In [7]:
len(c_unique)

300

In [8]:
c_unique

{'ad2007_lola',
 'ad2007_paul',
 'ad2007_sophia',
 'ad2007_will',
 'african_church_dreams1',
 'african_church_dreams2',
 'african_church_dreams3',
 'anna_kingsford_series',
 'anthropological_dreams',
 'bea_series',
 'betty_series',
 'beverly_1986',
 'beverly_1996',
 'beverly_2006',
 'beverly_2016',
 'brianna1',
 'brianna2',
 'brianna3',
 'brianna_journal',
 'bs_250',
 'bt_2013',
 'carl_series',
 'doug_series',
 'dreambank_barack',
 'dreambank_hilary',
 'dreamboarda',
 'dreamsfield',
 'ella_series',
 'film_dreams',
 'gackenbach',
 'gackenbach_nightmares_2010_107',
 'gackenbach_nightmares_2010_116',
 'gackenbach_nightmares_2010_124',
 'gackenbach_nightmares_2010_13',
 'gackenbach_nightmares_2010_131',
 'gackenbach_nightmares_2010_17',
 'gackenbach_nightmares_2010_2',
 'gackenbach_nightmares_2010_22',
 'gackenbach_nightmares_2010_26',
 'gackenbach_nightmares_2010_3',
 'gackenbach_nightmares_2010_37',
 'gackenbach_nightmares_2010_44',
 'gackenbach_nightmares_2010_49',
 'gackenbach_nightmar

### Clean Text, Remove stop words, Tokenize text, Apply LDA

In [11]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    


doc_set = df_rels["text"]
# list for tokenized documents in loop
texts = []

start_time = time.time()
nltk_errors_cnt = 0
# loop through document list
for ind,d in enumerate(doc_set):
    
    # clean and tokenize document string
    #raw = raw.translate(None, string.punctuation)
    raw = d.lower()
    
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    try:
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    except:
        print "doc", d
        print "stopped tokens: ", stopped_tokens
        nltk_errors_cnt += 1
        continue
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
print "Execution time: ", (time.time()-start_time)/60.0, " minutes."

ImportError: No module named stop_words

In [50]:
start_time = time.time()
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word = dictionary, passes=20)

print "Execution time: ", (time.time()-start_time)/60.0, " minutes."

Execution time:  67.319705534  minutes.


### Look into topics


In [51]:
for topic in ldamodel.print_topics(num_topics=50, num_words=8):
    print "Topic ", topic[0]
    print topic[1]

Topic  0
0.057*like + 0.048*someth + 0.029*think + 0.028*s + 0.015*also + 0.015*look + 0.014*one + 0.014*kind
Topic  1
0.040*photo + 0.029*french + 0.024*puppi + 0.024*greet + 0.021*weight + 0.021*tan + 0.019*50 + 0.018*pound
Topic  2
0.194*dream + 0.080*rememb + 0.021*time + 0.019*woke + 0.018*just + 0.015*last + 0.014*night + 0.014*realli
Topic  3
0.050*heart + 0.045*problem + 0.042*zombi + 0.039*film + 0.032*toilet + 0.023*castl + 0.021*tube + 0.019*medic
Topic  4
0.058*card + 0.043*island + 0.028*trump + 0.025*interview + 0.024*recal + 0.022*villag + 0.021*ladder + 0.019*jack
Topic  5
0.087*fire + 0.025*burn + 0.022*unknown + 0.021*worker + 0.017*tornado + 0.015*hate + 0.014*world + 0.014*none
Topic  6
0.048*dont + 0.043*im + 0.020*didnt + 0.020*cant + 0.019*goodby + 0.011*that + 0.008*id + 0.008*bull
Topic  7
0.120*friend + 0.051*girl + 0.026*talk + 0.023*boy + 0.022*boyfriend + 0.018*school + 0.017*us + 0.017*one
Topic  8
0.031*john + 0.025*fellow + 0.024*desert + 0.023*sun + 0.0

In [45]:
a = "491.  Wine high  Hard to remember how it started, over all the dream was searching for someone, or someplace, a lot took place in a grocery store. Everyone was stocking up, aed disaster was coming, people warn you to stay away from downtown. I take Emily up the coast to a high spot away from water. We follow the coast gliding our hands along the waves. At one point I end up paying 80 for a hotel but end up sleeping in a tent with 3 gay men. I get up to leave and go to the bar, where some guy I am dating is waiting for me. I explain to him that I have to leave, there is a man with a box telling me that I have a wish. I have a hard time thinking of what it should be."

In [47]:
tokens = tokenizer.tokenize(a)
stopped_tokens = [i for i in tokens if not i in en_stop]

In [52]:
#stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]